In [1]:
print("Welcome to the CarAnalyzer App!")

Welcome to the CarAnalyzer App!


In [2]:
import pandas as pd

In [3]:
cars_df = pd.read_csv('cars.csv')

In [4]:
cars_df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,205,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [5]:
### Data Exploration 
cars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [6]:
# checking for missing values
cars_df.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [7]:
# Enabling access to Jupyer Widgets
try:
    import piplite
    await piplite.install(['ipywidgets'])
except ImportError:
    pass

In [8]:
import ipywidgets as widgets

In [9]:
# connecting to MySQL database
import pymysql
host = "localhost"
user = "root"
password = "Raj121901!"
db = "cars"
db = pymysql.connect(host="localhost",user="root",password="Raj121901!",db="cars" )
cur = db.cursor()
cur.execute("SELECT * FROM sample_table")
output = cur.fetchall()

In [10]:
# list containing the names of the files containing the partitioned files 
partitionedDataFileNames = []
# partitioning the cars.csv data and saving each partition into a MySQL table
def csvPartitionData(csv_file_path, numPartitions): 
    # call the helper method to partition the data 
    partitionedDataFileNames.clear()
    partitionData(csv_file_path, numPartitions)
    print(partitionedDataFileNames)
    for file in partitionedDataFileNames:
        print(file)
        savePartitionedToSQL(file)

In [11]:
# helper function to partition cars.csv
def partitionData(csv_file_path, numPartitions): 
    # read the intial CSV file
    cars_df = pd.read_csv(csv_file_path)
    
    # partition into different CSV files 
    for i,chunk in enumerate(pd.read_csv('cars.csv', chunksize=len(cars_df)//numPartitions + 1)):
        new_file_name = 'chunk{}.csv'.format(i)
        chunk.to_csv(new_file_name, index=False)
        partitionedDataFileNames.append(new_file_name)

In [12]:
# helper method to save each partition as a table in MySQL
def savePartitionedToSQL(partitioned_csv_file): 
    for i,chunk in enumerate(pd.read_csv(partitioned_csv_file, chunksize=1)):
        car_ID = int(chunk["car_ID"])
        symboling = chunk["symboling"].to_string(index=False)
        CarName = chunk["CarName"].to_string(index=False)
        
        fueltype = chunk["fueltype"].to_string(index=False)
        aspiration = chunk["aspiration"].to_string(index=False)
        doornumber = chunk["doornumber"].to_string(index=False)
        carbody = chunk["carbody"].to_string(index=False)
        drivewheel = chunk["drivewheel"].to_string(index=False)
        enginelocation = chunk["enginelocation"].to_string(index=False)
        
        wheelbase = float(chunk["wheelbase"])
        carlength = float(chunk["carlength"])
        carwidth = float(chunk["carwidth"])
        carheight = float(chunk["carheight"])
        curbweight = int(chunk["curbweight"])
        
        enginetype = chunk["enginetype"].to_string(index=False)
        cylindernumber = chunk["cylindernumber"].to_string(index=False)
        
        enginesize = int(chunk["enginesize"])
        
        fuelsystem = chunk["fuelsystem"].to_string(index=False)
        
        boreratio = float(chunk["boreratio"])
        stroke = float(chunk["stroke"])
        compressionratio = float(chunk["compressionratio"])
        horsepower = int(chunk["horsepower"])
        peakrpm = int(chunk["peakrpm"])
        citympg = int(chunk["citympg"])
        highwaympg = int(chunk["highwaympg"])
        price = float(chunk["price"])
        
        table_name = partitioned_csv_file[0:6]
        if (i == 0):
            dropAndCreateTable(table_name)
        
        # insert each row of data 
        sql = '''INSERT INTO ''' + table_name + ''' (car_ID, symboling, CarName, fueltype, aspiration, doornumber, 
                             carbody, drivewheel, enginelocation, wheelbase, carlength, carwidth, 
                             carheight, curbweight, enginetype, cylindernumber, enginesize, fuelsystem, boreratio, 
                             stroke, compressionratio, horsepower, peakrpm, citympg, highwaympg, price) 
                             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
                             %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
        cur.execute(sql, (car_ID, symboling, CarName, fueltype, aspiration, doornumber, 
                          carbody, drivewheel, enginelocation, wheelbase, carlength, carwidth,
                          carheight, curbweight, enginetype, cylindernumber, enginesize, fuelsystem, boreratio, 
                          stroke, compressionratio, horsepower, peakrpm, citympg, highwaympg, price))
        db.commit()

In [13]:
# function that drops and creates the necessary table depending on the number of partitions 
def dropAndCreateTable(table_name):
    cur.execute("DROP TABLE IF EXISTS " + table_name)

    cur.execute("CREATE TABLE IF NOT EXISTS " + table_name + ''' (
    car_ID int, 
    symboling int, 
    CarName varchar(100),
    fueltype varchar(10), 
    aspiration varchar(10), 
    doornumber varchar(10), 
    carbody varchar(25), 
    drivewheel varchar(8), 
    enginelocation varchar(8), 
    wheelbase DECIMAL(4, 1), 
    carlength DECIMAL(4, 1), 
    carwidth DECIMAL(3, 1), 
    carheight DECIMAL(3, 1), 
    curbweight int, 
    enginetype varchar(8), 
    cylindernumber varchar(8), 
    enginesize int, 
    fuelsystem varchar(8), 
    boreratio DECIMAL(3, 2), 
    stroke DECIMAL(3, 2), 
    compressionratio DECIMAL(4, 2), 
    horsepower int, 
    peakrpm int, 
    citympg int, 
    highwaympg int, 
    price DECIMAL(8, 3), 
    PRIMARY KEY (car_ID)
    )''')

    db.commit()

In [14]:
# checking to ensure partition was sucessful 
csv_file_path = 'cars.csv'
numPartitions = 2
csvPartitionData(csv_file_path, numPartitions)

['chunk0.csv', 'chunk1.csv']
chunk0.csv
chunk1.csv


In [15]:
# Partitioning Function(s)

In [16]:
### mapPartition(p)
# list that will contain the results after executing mapPartition()
resultsList = []
# mapPartition() function to map a SQL statement to each partition
def mapPartition(p, sqlStatementSelect, sqlStatementFilters, displayInputAndOutput):
    table_name = p[0:6]
    sqlStatement = sqlStatementSelect + " FROM " + table_name + " " + sqlStatementFilters
    
    cur.execute(sqlStatement)
    records = cur.fetchall()
    
    if records:
        resultsList.append(records)
    else: 
        resultsList.append(False)
    
    if (displayInputAndOutput):
        print("Displaying the input for the mapPartition() function...")
        print("The function arguments are: \nthe name of the file,", p, "\nthe SELECT clause of the SQL statement", 
             sqlStatementSelect, "\nthe filter clause(s) of the SQL statement", sqlStatementFilters, 
              "\nand a boolean value of " + str(displayInputAndOutput) + " on whether or not to display the inputs and outputs\n")
        
        print("Displaying the output for the mapPartition() function...")
        print("The output from the SQL statement execution is: ", records)
        print("The list with the output from each partition is:")
        for resultIndex in range(len(resultsList)):
            print("This is element", resultIndex, "in the combined output list: ", resultsList[resultIndex])
        
        print("\033[1;3mThese are the inputs and outputs of this call to mapPartition() \033[0m \n")
    

In [17]:
### Reduce Functions

In [18]:
### Reduce functions
# reduce function for summing COUNT(*) from each partition when using GROUP BY statements 
def reduceGroupByCount(resultsList, displayInputAndOutput):
    resultsDict = {}
    for result in resultsList: 
        for record in result:
            if record[0] not in resultsDict.keys():
                resultsDict[record[0]] = record[1]
            else: 
                total_count = resultsDict.get(record[0])
                total_count += record[1]
                resultsDict[record[0]] = total_count
    
    if (displayInputAndOutput):
        print("Displaying the input for the reduceGroupByCount() function...")
        print("The function's arguments are a boolean value of", displayInputAndOutput,'''on whether or not
to display the inputs and outputs and the list with the output from each partition:''')
        for resultIndex in range(len(resultsList)):
            print("This is element", resultIndex, "in the combined output list: ", resultsList[resultIndex])

        print("\nDisplaying the output for the reduceGroupByCount() function...")
        print('''The output of this reduce function is a dictionary where the keys are the keys of the GROUP BY 
attribute/column value and the values are the number of occurences of each of the keys across all the partitions:''')
        for columnValue, numOccurences in resultsDict.items():
            print("A key is", "\'" + str(columnValue) + "\'", "and the number of cars with this value is", numOccurences)

        print("\033[1;3mThese are the inputs and outputs of this call to reduceGroupByCount() \033[0m \n")

    return resultsDict

In [19]:
# reduce function to sum the total from each partition
def reduceSumTotal(resultsList, displayInputAndOutput):
    totalNum = 0
    for result in resultsList:
        if result:
            totalNum += result[0][0]
            
    if (displayInputAndOutput):
        print("Displaying the input for the reduceSumTotal() function...")
        print("The function's arguments are a boolean value of", displayInputAndOutput,'''on whether or not
to display the inputs and outputs and the list with the output from each partition:''')
        for resultIndex in range(len(resultsList)):
            print("This is element", resultIndex, "in the combined output list: ", resultsList[resultIndex])

        print("\nDisplaying the output for the reduceSumTotal() function...")
        print("The output is the sum of the totals from each partition.")
        print("The sum outputted by this call to the reduce function is " + str(totalNum) + ".")

        print("\033[1;3mThese are the inputs and outputs of this call to reduceSumTotal() \033[0m \n")
    
    return totalNum

In [20]:
# reduce function to identify the max or min of the entire dataset by comparing values from each partition
def reduceMaxOrMin(resultsList, typeOfComparison, displayInputAndOutput):
    listOfPrices = []
    returnedIndexValue = 0
    if resultsList: 
        for result in resultsList:
            listOfPrices.append(result[0][1])

    if typeOfComparison == "MAX":
        maxValue = listOfPrices[0]
        maxValueIndex = 0
        for priceIndex in range(len(listOfPrices)): 
            if (listOfPrices[priceIndex] > maxValue): 
                maxValue = listOfPrices[priceIndex]
                maxValueIndex = priceIndex
        returnedIndexValue = maxValueIndex
          
        
    else:
        for price in listOfPrices:
            minValue = listOfPrices[0]
            minValueIndex = 0
            for priceIndex in range(len(listOfPrices)): 
                if (listOfPrices[priceIndex] < minValue): 
                    minValue = listOfPrices[priceIndex]
                    minValueIndex = priceIndex
        returnedIndexValue = minValueIndex
        
    if (displayInputAndOutput):
        print("Displaying the input for the reduceMaxOrMin() function...")
        print("The function's arguments are:\na boolean value of " + str(displayInputAndOutput) + '''on whether or not
to display the inputs and outputs''' + "\na string", typeOfComparison, '''identifying whether the user would like to 
find the Maximum or Minimum''' + '''\nand the list with the output from each partition:''')
        for resultIndex in range(len(resultsList)):
            print("This is element", resultIndex, "in the combined output list: ", resultsList[resultIndex])

        print("\nDisplaying the output for the reduceMaxOrMin() function...")
        print('''The output is the index value corresponding to the greatest or least value of the corresponding 
maximum or minimum value from each partition.''')
        print("The index outputted by this call to the reduce function is " + str(returnedIndexValue) + ".")

        print("\033[1;3mThese are the inputs and outputs of this call to reduceMaxOrMin() \033[0m \n")
    
    return returnedIndexValue

In [21]:
# reduce function that appends the list of values from each of the partitions to output one unified list of values
def reduceListOfValues(resultsList, displayInputAndOutput):
    listOfCarIDs = []
    for result in resultsList:
        for car_ID in result:
            if car_ID not in listOfCarIDs:
                listOfCarIDs.append(car_ID[0])
                
    if (displayInputAndOutput):
        print("Displaying the input for the reduceListOfValues() function...")
        print("The function's arguments are a boolean value of", displayInputAndOutput,'''on whether or not
to display the inputs and outputs and the list with the output from each partition:''')
        for resultIndex in range(len(resultsList)):
            print("This is element", resultIndex, "in the combined output list: ", resultsList[resultIndex])

        print("\nDisplaying the output for the reduceListOfValues() function...")
        print("The output is the combined list of values/carIDs from each partition.")
        for carIDIndex in range(len(listOfCarIDs)): 
            print("The carID of element", carIDIndex, "is", listOfCarIDs[carIDIndex])

        print("\033[1;3mThese are the inputs and outputs of this call to reduceListOfValues() \033[0m \n")
        
    return listOfCarIDs

In [22]:
### Search and Analytics Functions

In [23]:
### Finding the number of different carbodies whose fueltype is gas 
# Get user input on whether or not to show inputs and outputs of map and reduce functions
displayInfo = False
displayInputsAndOutputs = widgets.ToggleButtons(
    options=['Yes', 'No'],
    description='Display?',
    disabled=False,
    style=dict(
        font_weight='bold'
    ),
    button_style='warning', 
)
display(displayInputsAndOutputs)

ToggleButtons(button_style='warning', description='Display?', options=('Yes', 'No'), style=ToggleButtonsStyle(…

In [24]:
# convert the user input on displaying inputs and outputs to boolean values 
if (displayInputsAndOutputs.value == "Yes"):
    displayInfo = True
else: 
    displayInfo = False

def numDifferentCarBodies():
    resultsList.clear()
    for file in partitionedDataFileNames:
        sqlStatement1 = "SELECT carBody, COUNT(*)"
        sqlStatement2 = "WHERE fueltype = 'gas' GROUP BY carBody"
        mapPartition(file, sqlStatement1, sqlStatement2, displayInfo)

    reducedDict = reduceGroupByCount(resultsList, displayInfo)
    for carbody, numCars in reducedDict.items():
        print("There are", numCars, carbody + "s")
    
numDifferentCarBodies()

Displaying the input for the mapPartition() function...
The function arguments are: 
the name of the file, chunk0.csv 
the SELECT clause of the SQL statement SELECT carBody, COUNT(*) 
the filter clause(s) of the SQL statement WHERE fueltype = 'gas' GROUP BY carBody 
and a boolean value of True on whether or not to display the inputs and outputs

Displaying the output for the mapPartition() function...
The output from the SQL statement execution is:  (('convertible', 3), ('hatchback', 39), ('sedan', 46), ('wagon', 6), ('hardtop', 2))
The list with the output from each partition is:
This is element 0 in the combined output list:  (('convertible', 3), ('hatchback', 39), ('sedan', 46), ('wagon', 6), ('hardtop', 2))
These are the inputs and outputs of this call to mapPartition()  

Displaying the input for the mapPartition() function...
The function arguments are: 
the name of the file, chunk1.csv 
the SELECT clause of the SQL statement SELECT carBody, COUNT(*) 
the filter clause(s) of the 

In [25]:
### Finding the number of different cars with certain characteristics with a specific fueltype or other attribute
# Get user input on whether or not to show inputs and outputs of map and reduce functions
displayInfo = False
displayInputsAndOutputs = widgets.ToggleButtons(
    options=['Yes', 'No'],
    description='Display?',
    disabled=False,
    style=dict(
        font_weight='bold'
    ),
    button_style='warning', 
)
display(displayInputsAndOutputs)

ToggleButtons(button_style='warning', description='Display?', options=('Yes', 'No'), style=ToggleButtonsStyle(…

In [26]:
# Get the characteristic the user would like to group by 
selectCharacteristicDropdown = widgets.Dropdown(
    options=["carbody", "enginelocation", "cylindernumber"],
    value="carbody",
    description='Select:',
    disabled=False,
)
display(selectCharacteristicDropdown)

Dropdown(description='Select:', options=('carbody', 'enginelocation', 'cylindernumber'), value='carbody')

In [27]:
# Get the characteristic the user would like to filter data based on and display the corresponding potential value 
# for each characteristic in the dropdowns below 
filterCharacteristicDropdown = widgets.Dropdown(
    options=["fueltype", "aspiration", "enginetype"],
    value="fueltype",
    description='Filter Attr:',
    disabled=False,
)
display(filterCharacteristicDropdown)

Dropdown(description='Filter Attr:', options=('fueltype', 'aspiration', 'enginetype'), value='fueltype')

In [28]:
if (filterCharacteristicDropdown.value == "fueltype"):
    fueltypeDropdown = widgets.Dropdown(
        options=["gas", "diesel"],
        value="gas",
        description='Fuel Type:',
        disabled=False,
    )
    display(fueltypeDropdown)
elif (filterCharacteristicDropdown.value == "aspiration"): 
    aspirationDropdown = widgets.Dropdown(
        options=["turbo", "std"],
        value="turbo",
        description='Aspiration:',
        disabled=False,
    )
    display(aspirationDropdown)
else: 
    enginetypeDropdown = widgets.Dropdown(
        options=["dohc", "ohcv", "ohc", "l", "rotor", "dohcv"],
        value="dohc",
        description='Engine Type:',
        disabled=False,
    )
    display(enginetypeDropdown)

Dropdown(description='Fuel Type:', options=('gas', 'diesel'), value='gas')

In [29]:
# convert the user input on displaying inputs and outputs to boolean values 
if (displayInputsAndOutputs.value == "Yes"):
    displayInfo = True
else: 
    displayInfo = False
    
# assign all the necessary, chosen dropdown values 
if (filterCharacteristicDropdown.value == "fueltype"):
    filterCharacteristicValue = fueltypeDropdown.value
elif (filterCharacteristicDropdown.value == "aspiration"):
    filterCharacteristicValue = aspirationDropdown.value
elif (filterCharacteristicDropdown.value == "enginetype"):
    filterCharacteristicValue = enginetypeDropdown.value

selectCharacteristic = selectCharacteristicDropdown.value
filterCharacteristic = filterCharacteristicDropdown.value


def numDifferentCarWithCustomSelectAndFilter(selectCharacteristic, filterCharacteristic, filterCharacteristicValue):
    resultsList.clear()
    for file in partitionedDataFileNames:
        sqlStatement1 = "SELECT " + selectCharacteristic + ", COUNT(*)"
        sqlStatement2 = "WHERE " + filterCharacteristic + " = " + "\'" + \
                        filterCharacteristicValue + "\' GROUP BY " + selectCharacteristic
        mapPartition(file, sqlStatement1, sqlStatement2, displayInfo)

    reducedDict = reduceGroupByCount(resultsList, displayInfo)
    for carbody, numCars in reducedDict.items():
        print("There are", numCars, carbody + "s")
        
numDifferentCarWithCustomSelectAndFilter(selectCharacteristic, filterCharacteristic, filterCharacteristicValue)

Displaying the input for the mapPartition() function...
The function arguments are: 
the name of the file, chunk0.csv 
the SELECT clause of the SQL statement SELECT carbody, COUNT(*) 
the filter clause(s) of the SQL statement WHERE fueltype = 'gas' GROUP BY carbody 
and a boolean value of True on whether or not to display the inputs and outputs

Displaying the output for the mapPartition() function...
The output from the SQL statement execution is:  (('convertible', 3), ('hatchback', 39), ('sedan', 46), ('wagon', 6), ('hardtop', 2))
The list with the output from each partition is:
This is element 0 in the combined output list:  (('convertible', 3), ('hatchback', 39), ('sedan', 46), ('wagon', 6), ('hardtop', 2))
These are the inputs and outputs of this call to mapPartition()  

Displaying the input for the mapPartition() function...
The function arguments are: 
the name of the file, chunk1.csv 
the SELECT clause of the SQL statement SELECT carbody, COUNT(*) 
the filter clause(s) of the 

In [30]:
### Finding the number of cars within a certain price range 
### Finding the number of different cars with certain characteristics with a specific fueltype or other attribute
# Get user input on whether or not to show inputs and outputs of map and reduce functions
displayInfo = False
displayInputsAndOutputs = widgets.ToggleButtons(
    options=['Yes', 'No'],
    description='Display?',
    disabled=False,
    style=dict(
        font_weight='bold'
    ),
    button_style='warning', 
)
display(displayInputsAndOutputs)

ToggleButtons(button_style='warning', description='Display?', options=('Yes', 'No'), style=ToggleButtonsStyle(…

In [31]:
# First get the lower bound price from the user 
priceLowerBound = widgets.FloatSlider(
    value=15000,
    min=0,
    max=45400,
    step=0.1,
    description='Lower Price:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(priceLowerBound)

FloatSlider(value=15000.0, description='Lower Price:', max=45400.0, readout_format='d')

In [32]:
# Using the lower bound price as the minimum for the upper bound price, get the upper bound price from the user 
priceUpperBound = widgets.FloatSlider(
    value=priceLowerBound.value + 1000,
    min=priceLowerBound.value,
    max=45400,
    step=1,
    description='Upper Price:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(priceUpperBound)

FloatSlider(value=16000.0, continuous_update=False, description='Upper Price:', max=45400.0, min=15000.0, read…

In [33]:
# Assign the user-selected values
minPrice = priceLowerBound.value
maxPrice = priceUpperBound.value

# convert the user input on displaying inputs and outputs to boolean values 
if (displayInputsAndOutputs.value == "Yes"):
    displayInfo = True
else: 
    displayInfo = False

def numCarsWithinPriceRange(minPrice, maxPrice):
    resultsList.clear()
    for file in partitionedDataFileNames: 
        sqlStatement1 = "SELECT COUNT(*)"
        sqlStatement2 = "WHERE price >= " + str(minPrice) + " AND price <= " + str(maxPrice)
        mapPartition(file, sqlStatement1, sqlStatement2, displayInfo)

    totalNum = reduceSumTotal(resultsList, displayInfo)
    print("There are", totalNum, "cars within the price range of", '{:.2f}'.format(minPrice), 
          "and", '{:.2f}'.format(maxPrice), "dollars")

numCarsWithinPriceRange(minPrice, maxPrice)

Displaying the input for the mapPartition() function...
The function arguments are: 
the name of the file, chunk0.csv 
the SELECT clause of the SQL statement SELECT COUNT(*) 
the filter clause(s) of the SQL statement WHERE price >= 15000.0 AND price <= 16000.0 
and a boolean value of True on whether or not to display the inputs and outputs

Displaying the output for the mapPartition() function...
The output from the SQL statement execution is:  ((2,),)
The list with the output from each partition is:
This is element 0 in the combined output list:  ((2,),)
These are the inputs and outputs of this call to mapPartition()  

Displaying the input for the mapPartition() function...
The function arguments are: 
the name of the file, chunk1.csv 
the SELECT clause of the SQL statement SELECT COUNT(*) 
the filter clause(s) of the SQL statement WHERE price >= 15000.0 AND price <= 16000.0 
and a boolean value of True on whether or not to display the inputs and outputs

Displaying the output for th

In [34]:
### Find the maximum or minimum price of a car with a certain characteristic 
# Get user input on whether or not to show inputs and outputs of map and reduce functions
displayInfo = False
displayInputsAndOutputs = widgets.ToggleButtons(
    options=['Yes', 'No'],
    description='Display?',
    disabled=False,
    style=dict(
        font_weight='bold'
    ),
    button_style='warning', 
)
display(displayInputsAndOutputs)

ToggleButtons(button_style='warning', description='Display?', options=('Yes', 'No'), style=ToggleButtonsStyle(…

In [35]:
# Determine whether the user would like to find the maximum or minimum price 
minOrMaxButton = widgets.ToggleButtons(
    options=['Minimum', 'Maximum'],
    description='Min/Max:',
    disabled=False,
    button_style='info', 
)
display(minOrMaxButton)

ToggleButtons(button_style='info', description='Min/Max:', options=('Minimum', 'Maximum'), value='Minimum')

In [36]:
# Get the characteristic the user would like to filter data based on and display the corresponding potential value 
# for each characteristic in the dropdowns below 
filterCharacteristicDropdown = widgets.Dropdown(
    options=["fueltype", "aspiration", "enginetype"],
    value="fueltype",
    description='Filter Attr:',
    disabled=False,
)
display(filterCharacteristicDropdown)

Dropdown(description='Filter Attr:', options=('fueltype', 'aspiration', 'enginetype'), value='fueltype')

In [37]:
if (filterCharacteristicDropdown.value == "fueltype"):
    fueltypeDropdown = widgets.Dropdown(
        options=["gas", "diesel"],
        value="gas",
        description='Fuel Type:',
        disabled=False,
    )
    display(fueltypeDropdown)
elif (filterCharacteristicDropdown.value == "aspiration"): 
    aspirationDropdown = widgets.Dropdown(
        options=["turbo", "std"],
        value="turbo",
        description='Aspiration:',
        disabled=False,
    )
    display(aspirationDropdown)
else: 
    enginetypeDropdown = widgets.Dropdown(
        options=["dohc", "ohcv", "ohc", "l", "rotor", "dohcv"],
        value="dohc",
        description='Engine Type:',
        disabled=False,
    )
    display(enginetypeDropdown)

Dropdown(description='Fuel Type:', options=('gas', 'diesel'), value='gas')

In [38]:
# convert the user input on displaying inputs and outputs to boolean values 
if (displayInputsAndOutputs.value == "Yes"):
    displayInfo = True
else: 
    displayInfo = False

# assign all the necessary, chosen dropdown values 
if (filterCharacteristicDropdown.value == "fueltype"):
    filterCharacteristicValue = fueltypeDropdown.value
elif (filterCharacteristicDropdown.value == "aspiration"):
    filterCharacteristicValue = aspirationDropdown.value
elif (filterCharacteristicDropdown.value == "enginetype"):
    filterCharacteristicValue = enginetypeDropdown.value

filterCharacteristic = filterCharacteristicDropdown.value

# need to assign "MAX" or "MIN", not "Maximum" or "Minimum", because MySQL uses "MAX" or "MIN" when querying
if (minOrMaxButton.value == "Maximum"):
    minOrMax = "MAX"
else: 
    minOrMax = "MIN"

def carMaxOrMinPrice(maxOrMin, filterCharacteristic, filterCharacteristicValue):
    resultsList.clear()
    for file in partitionedDataFileNames: 
        if maxOrMin == "MAX":
            sqlStatement1 = "SELECT CarName, price"
            sqlStatement2 = "WHERE price >= (SELECT " + maxOrMin + "(price) FROM " + file[0:6] + \
                            " WHERE " + filterCharacteristic + " = \'" + filterCharacteristicValue + "\')"
            mapPartition(file, sqlStatement1, sqlStatement2, displayInfo)

        else: 
            sqlStatement1 = "SELECT CarName, price"
            sqlStatement2 = "WHERE price <= (SELECT " + maxOrMin + "(price) FROM " + file[0:6] + \
                            " WHERE " + filterCharacteristic + " = \'" + filterCharacteristicValue + "\')"
            mapPartition(file, sqlStatement1, sqlStatement2, displayInfo)

    reducedResult = reduceMaxOrMin(resultsList, maxOrMin, displayInfo)
    if maxOrMin == "MAX":
        print("The name of the car with the highest price of " + '{:.2f}'.format(resultsList[reducedResult][0][1]) + \
              " dollars is the " + resultsList[reducedResult][0][0])
    else:
        print("The name of the car with the lowest price of " + '{:.2f}'.format(resultsList[reducedResult][0][1]) + \
              " dollars is the " + resultsList[reducedResult][0][0])

carMaxOrMinPrice(minOrMax, filterCharacteristic, filterCharacteristicValue)

Displaying the input for the mapPartition() function...
The function arguments are: 
the name of the file, chunk0.csv 
the SELECT clause of the SQL statement SELECT CarName, price 
the filter clause(s) of the SQL statement WHERE price <= (SELECT MIN(price) FROM chunk0 WHERE fueltype = 'gas') 
and a boolean value of True on whether or not to display the inputs and outputs

Displaying the output for the mapPartition() function...
The output from the SQL statement execution is:  (('chevrolet impala', Decimal('5151.000')),)
The list with the output from each partition is:
This is element 0 in the combined output list:  (('chevrolet impala', Decimal('5151.000')),)
These are the inputs and outputs of this call to mapPartition()  

Displaying the input for the mapPartition() function...
The function arguments are: 
the name of the file, chunk1.csv 
the SELECT clause of the SQL statement SELECT CarName, price 
the filter clause(s) of the SQL statement WHERE price <= (SELECT MIN(price) FROM chu

In [39]:
### Find all the car_IDs with a certain characteristic 
# Get user input on whether or not to show inputs and outputs of map and reduce functions
displayInfo = False
displayInputsAndOutputs = widgets.ToggleButtons(
    options=['Yes', 'No'],
    description='Display?',
    disabled=False,
    style=dict(
        font_weight='bold'
    ),
    button_style='warning', 
)
display(displayInputsAndOutputs)

ToggleButtons(button_style='warning', description='Display?', options=('Yes', 'No'), style=ToggleButtonsStyle(…

In [40]:
# Get the characteristic the user would like to filter data based on and display the corresponding potential value 
# for each characteristic in the dropdowns below 
filterCharacteristicDropdown = widgets.Dropdown(
    options=["fueltype", "aspiration", "enginetype"],
    value="fueltype",
    description='Filter Attr:',
    disabled=False,
)
display(filterCharacteristicDropdown)

Dropdown(description='Filter Attr:', options=('fueltype', 'aspiration', 'enginetype'), value='fueltype')

In [41]:
if (filterCharacteristicDropdown.value == "fueltype"):
    fueltypeDropdown = widgets.Dropdown(
        options=["gas", "diesel"],
        value="gas",
        description='Fuel Type:',
        disabled=False,
    )
    display(fueltypeDropdown)
elif (filterCharacteristicDropdown.value == "aspiration"): 
    aspirationDropdown = widgets.Dropdown(
        options=["turbo", "std"],
        value="turbo",
        description='Aspiration:',
        disabled=False,
    )
    display(aspirationDropdown)
else: 
    enginetypeDropdown = widgets.Dropdown(
        options=["dohc", "ohcv", "ohc", "l", "rotor", "dohcv"],
        value="dohc",
        description='Engine Type:',
        disabled=False,
    )
    display(enginetypeDropdown)

Dropdown(description='Fuel Type:', options=('gas', 'diesel'), value='gas')

In [42]:
# convert the user input on displaying inputs and outputs to boolean values 
if (displayInputsAndOutputs.value == "Yes"):
    displayInfo = True
else: 
    displayInfo = False
    
# assign all the necessary, chosen dropdown values 
if (filterCharacteristicDropdown.value == "fueltype"):
    filterCharacteristicValue = fueltypeDropdown.value
elif (filterCharacteristicDropdown.value == "aspiration"):
    filterCharacteristicValue = aspirationDropdown.value
elif (filterCharacteristicDropdown.value == "enginetype"):
    filterCharacteristicValue = enginetypeDropdown.value

filterCharacteristic = filterCharacteristicDropdown.value
    
def findCarIDs(filterCharacteristic, filterCharacteristicValue):
    resultsList.clear()
    for file in partitionedDataFileNames: 
        sqlStatement1 = "SELECT car_ID"
        sqlStatement2 = "WHERE " + filterCharacteristic + " = \'" + filterCharacteristicValue + "\'"
        mapPartition(file, sqlStatement1, sqlStatement2, displayInfo)

    listOfCarIDs = reduceListOfValues(resultsList, displayInfo)
    if listOfCarIDs:
        print("The car IDs with " + filterCharacteristic + " equal to " + filterCharacteristicValue + " are:")
        for car_ID in listOfCarIDs:
            print(car_ID)
        print("There is/are " + str(len(listOfCarIDs)) + " car ID(s) in total!")
    else:
        print("No car IDs with " + filterCharacteristic + " equal to " + filterCharacteristicValue)
        print("Please try another value!")

findCarIDs(filterCharacteristic, filterCharacteristicValue)

Displaying the input for the mapPartition() function...
The function arguments are: 
the name of the file, chunk0.csv 
the SELECT clause of the SQL statement SELECT car_ID 
the filter clause(s) of the SQL statement WHERE fueltype = 'gas' 
and a boolean value of True on whether or not to display the inputs and outputs

Displaying the output for the mapPartition() function...
The output from the SQL statement execution is:  ((2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,), (13,), (14,), (15,), (16,), (17,), (18,), (19,), (20,), (21,), (22,), (23,), (24,), (25,), (26,), (27,), (28,), (29,), (30,), (31,), (32,), (33,), (34,), (35,), (36,), (37,), (38,), (39,), (40,), (41,), (42,), (43,), (44,), (45,), (46,), (47,), (48,), (49,), (50,), (51,), (52,), (53,), (54,), (55,), (56,), (57,), (58,), (59,), (60,), (61,), (62,), (63,), (65,), (66,), (72,), (73,), (74,), (75,), (76,), (77,), (78,), (79,), (80,), (81,), (82,), (83,), (84,), (85,), (86,), (87,), (88,), (89,), (90,), 

In [43]:
### Find the CarName corresponding to a specific car_ID 
# Get user input on whether or not to show inputs and outputs of map and reduce functions
displayInfo = False
displayInputsAndOutputs = widgets.ToggleButtons(
    options=['Yes', 'No'],
    description='Display?',
    disabled=False,
    style=dict(
        font_weight='bold'
    ),
    button_style='warning', 
)
display(displayInputsAndOutputs)

ToggleButtons(button_style='warning', description='Display?', options=('Yes', 'No'), style=ToggleButtonsStyle(…

In [44]:
# Get user input on the car_ID value 
carIDTextBox = widgets.BoundedIntText(
    value=50,
    min=1,
    max=len(cars_df),
    step=1,
    description='Text:',
    disabled=False
)
display(carIDTextBox)

BoundedIntText(value=50, description='Text:', max=205, min=1)

In [45]:
# convert the user input on displaying inputs and outputs to boolean values 
if (displayInputsAndOutputs.value == "Yes"):
    displayInfo = True
else: 
    displayInfo = False
    
# Assign the user-entered value
car_ID = carIDTextBox.value

def findCarNameFromID(car_ID):
    resultsList.clear()
    for file in partitionedDataFileNames: 
        sqlStatement1 = "SELECT CarName"
        sqlStatement2 = "WHERE car_ID = \'" + str(car_ID) + "\'"
        mapPartition(file, sqlStatement1, sqlStatement2, displayInfo)
        
    if displayInfo: 
        print("The function findCarNameFromID() does not use/need a reduce function!")
        print("So no input or output related to a reduce function will be displayed.")
        print("\033[1;3mThese are the inputs and outputs of this call to findCarNameFromID() \033[0m \n")
        
    numFalseValues = 0
    for result in resultsList: 
        if result:
            carName = result[0][0]
            print("The car name corresponding to the car ID of " + str(car_ID) + " is " + carName + ".")
        else: 
            numFalseValues += 1
    # Display message if there are no corresponding car names 
    if (numFalseValues == 2):
        print("There are no car names corresponding to a car ID of " + str(car_ID))
        print("Please try again with another car ID value!")
            
findCarNameFromID(car_ID)

Displaying the input for the mapPartition() function...
The function arguments are: 
the name of the file, chunk0.csv 
the SELECT clause of the SQL statement SELECT CarName 
the filter clause(s) of the SQL statement WHERE car_ID = '50' 
and a boolean value of True on whether or not to display the inputs and outputs

Displaying the output for the mapPartition() function...
The output from the SQL statement execution is:  (('jaguar xk',),)
The list with the output from each partition is:
This is element 0 in the combined output list:  (('jaguar xk',),)
These are the inputs and outputs of this call to mapPartition()  

Displaying the input for the mapPartition() function...
The function arguments are: 
the name of the file, chunk1.csv 
the SELECT clause of the SQL statement SELECT CarName 
the filter clause(s) of the SQL statement WHERE car_ID = '50' 
and a boolean value of True on whether or not to display the inputs and outputs

Displaying the output for the mapPartition() function...
T

In [46]:
print("Thank you for taking the time to interact with this CarAnalyzer app!")

Thank you for taking the time to interact with this CarAnalyzer app!
